In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')

print(df.columns)
df.info()
df.describe()

/tmp/ipykernel_114732/3049508150.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|')


Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data 

,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,NumberOfVehiclesInFleet,SumInsured,CalculatedPremiumPerTerm,TotalPremium,TotalClaims
count,1.000098e+06,1.000098e+06,1.000098e+06,9.995460e+05,1.000098e+06,999546.000000,999546.000000,999546.000000,999546.000000,2.204560e+05,0.0,1.000098e+06,1.000098e+06,1.000098e+06,1.000098e+06
mean,1.048175e+05,7.956682e+03,3.020601e+03,5.487770e+07,2.010225e+03,4.046642,2466.743258,97.207919,4.019250,2.255311e+05,NaN,6.041727e+05,1.178757e+02,6.190550e+01,6.486119e+01
std,6.329371e+04,5.290039e+03,2.649854e+03,1.360381e+07,3.261391e+00,0.294020,442.800640,19.393256,0.468314,5.645157e+05,NaN,1.508332e+06,3.997017e+02,2.302845e+02,2.384075e+03
min,1.000000e+00,1.400000e+01,1.000000e+00,4.041200e+06,1.987000e+03,0.000000,0.000000,0.000000,0.000000,2.000000e+04,NaN,1.000000e-02,0.000000e+00,-7.825768e+02,-1.200241e+04
25%,5.514300e+04,4.500000e+03,8.270000e+02,6.005692e+07,2.008000e+03,4.000000,2237.000000,75.000000,4.000000,1.350000e+05,NaN,5.000000e+03,3.224800e+00,0.000000e+00,0.000000e+00
50%,9.408300e+04,7.071000e+03,2.000000e+03,6.005842e+07,2.011000e+03,4.000000,2694.000000,111.000000,4.000000,2.200000e+05,NaN,7.500000e+03,8.436900e+00,2.178333e+00,0.000000e+00
75%,1.391900e+05,1.107700e+04,4.180000e+03,6.005842e+07,2.013000e+03,4.000000,2694.000000,111.000000,4.000000,2.800000e+05,NaN,2.500000e+05,9.000000e+01,2.192982e+01,0.000000e+00
max,3.011750e+05,2.324600e+04,9.870000e+03,6.506535e+07,2.015000e+03,10.000000,12880.000000,309.000000,6.000000,2.655000e+07,NaN,1.263620e+07,7.442217e+04,6.528260e+04,3.930921e+05


### Create Metrics

In [2]:
df['ClaimOccurred'] = df['TotalClaims'] > 0
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

### No Risk Difference Across Provinces

In [3]:
from scipy.stats import ttest_ind, chi2_contingency

# Frequency: Chi-Square Test
province_freq = pd.crosstab(df['Province'], df['ClaimOccurred'])
chi2, p, _, _ = chi2_contingency(province_freq)
print("Province Frequency Chi-square p-value:", p)

# Severity: T-test for provinces with claims
claimed = df[df['ClaimOccurred']]
prov_groups = [g['TotalClaims'].values for n, g in claimed.groupby('Province') if len(g) > 20]
for i in range(len(prov_groups)-1):
    for j in range(i+1, len(prov_groups)):
        t, p = ttest_ind(prov_groups[i], prov_groups[j], equal_var=False)
        print(f"Group {i+1} vs Group {j+1} Severity p-value: {p}")

Province Frequency Chi-square p-value: 5.925510718204677e-19
Group 1 vs Group 2 Severity p-value: 0.5092044160965097
Group 1 vs Group 3 Severity p-value: 0.7429468731869167
Group 1 vs Group 4 Severity p-value: 0.1407523320574504
Group 1 vs Group 5 Severity p-value: 0.15457189472030985
Group 1 vs Group 6 Severity p-value: 0.18050271818657174
Group 1 vs Group 7 Severity p-value: 0.9004593358050077
Group 2 vs Group 3 Severity p-value: 0.000655809991172911
Group 2 vs Group 4 Severity p-value: 0.047887459586952126
Group 2 vs Group 5 Severity p-value: 0.026395472077451004
Group 2 vs Group 6 Severity p-value: 0.009582783721434732
Group 2 vs Group 7 Severity p-value: 0.03059896049292002
Group 3 vs Group 4 Severity p-value: 0.00030987596334938495
Group 3 vs Group 5 Severity p-value: 3.4788398243029936e-05
Group 3 vs Group 6 Severity p-value: 1.4801883458821668e-06
Group 3 vs Group 7 Severity p-value: 0.6315610124651586
Group 4 vs Group 5 Severity p-value: 0.8501546670512836
Group 4 vs Group 6 S

### No Risk Differences Between Zip Codes

In [4]:
# Check existence of column
if 'PostalCode' in df.columns:
    zip_freq = pd.crosstab(df['PostalCode'], df['ClaimOccurred'])
    chi2, p, _, _ = chi2_contingency(zip_freq)
    print("Postal Code Frequency Chi-square p-value:", p)

Postal Code Frequency Chi-square p-value: 3.152172246339057e-30


### No Margin Difference Between Zip Codes

In [6]:
from scipy.stats import f_oneway

zip_groups = [g['Margin'].values for _, g in df.groupby('PostalCode') if len(g) > 10]
f_stat, p = f_oneway(*zip_groups)
print("ANOVA p-value for Margin difference across zip codes:", p)

ANOVA p-value for Margin difference across zip codes: 0.970313591024769


### No Risk Difference Between Men and Women

In [7]:
# Frequency
gender_freq = pd.crosstab(df['Gender'], df['ClaimOccurred'])
chi2, p, _, _ = chi2_contingency(gender_freq)
print("Gender Frequency Chi-square p-value:", p)

# Severity
males = df[(df['ClaimOccurred']) & (df['Gender'] == 'Male')]['TotalClaims']
females = df[(df['ClaimOccurred']) & (df['Gender'] == 'Female')]['TotalClaims']
t, p = ttest_ind(males, females, equal_var=False)
print("Gender Severity T-test p-value:", p)

Gender Frequency Chi-square p-value: 0.026570248768437145
Gender Severity T-test p-value: 0.5680286951630678
